In [27]:
import fitsio as fio
import numpy as np
import os, sys
import glob
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm import tqdm
import pickle
from des_y6utils import mdet
%matplotlib inline

os.environ["MEDS_DIR"] = "/global/cfs/cdirs/des/y6-shear-catalogs"

In [7]:
# grab mdet files (these files are divided into patches)
mdet_files = glob.glob('/global/cfs/cdirs/des/y6-shear-catalogs/Y6A2_METADETECT_V5a/jackknife_patches_blinded/patch-*.fits')
# mdet_files = glob.glob('/global/cfs/cdirs/des/y6-shear-catalogs/Y6A2_METADETECT_V3/*_metadetect.fits')
# mdet_files = np.load('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/som_weight_test/som_training_files.npy')

In [3]:
# preliminary shear weight
with open('/pscratch/sd/m/myamamot/des-y6-analysis/y6_measurement/v3/inverse_variance_weight_v3_s2n_10-300_Tratio_0.5-5.pickle', 'rb') as handle:
    wgt_dict = pickle.load(handle)

In [4]:
def assign_loggrid(x, y, xmin, xmax, xsteps, ymin, ymax, ysteps):
    from math import log10
    # return x and y indices of data (x,y) on a log-spaced grid that runs from [xy]min to [xy]max in [xy]steps

    logstepx = log10(xmax/xmin)/xsteps
    logstepy = log10(ymax/ymin)/ysteps

    indexx = (np.log10(x/xmin)/logstepx).astype(int)
    indexy = (np.log10(y/ymin)/logstepy).astype(int)

    indexx = np.maximum(indexx,0)
    indexx = np.minimum(indexx, xsteps-1)
    indexy = np.maximum(indexy,0)
    indexy = np.minimum(indexy, ysteps-1)

    return indexx,indexy

def _find_shear_weight(d, wgt_dict, snmin, snmax, sizemin, sizemax, steps, mdet_mom):
    
    if wgt_dict is None:
        weights = np.ones(len(d))
        return weights

    shear_wgt = wgt_dict['weight']
    shear_response = wgt_dict['response']
    indexx, indexy = assign_loggrid(d[mdet_mom+'_s2n'], d[mdet_mom+'_T_ratio'], snmin, snmax, steps, sizemin, sizemax, steps)
    weights = np.array([shear_wgt[x, y] for x, y in zip(indexx, indexy)])
    response = np.array([shear_response[x, y] for x, y in zip(indexx, indexy)])
    
    return response, weights

In [8]:
f_response = open('/pscratch/sd/m/myamamot/des-y6-analysis/y6_measurement/v3/weighted_shear_response_v3_s2n_10-300_Tratio_0.5-5.txt', 'r')
R11, R22 = f_response.read().split('\n')
R = (float(R11) + float(R22))/2

f_response_shape_err = open('/pscratch/sd/m/myamamot/des-y6-analysis/y6_measurement/v5/weighted_shear_response_v5_shape_err.txt', 'r')
R11_se, R22_se = f_response_shape_err.read().split('\n')
R_shape_err = (float(R11_se) + float(R22_se))/2

In [9]:
print(R11, R22, R)
print(R11_se, R22_se, R_shape_err)

0.8435480631935742 0.8465420261413853 0.8450450446674798
0.7613651310846075 0.7646407933123135 0.7630029621984604


In [10]:
# Put all the objects in an array.
obj_list = []
def flux2mag(flux, zero_pt=30):
    return zero_pt - 2.5 * np.log10(flux)
for i,fname in tqdm(enumerate(mdet_files)):
    
    # try:
    d = fio.read(fname)
    msk = mdet.make_mdet_cuts(d, 5) 
    mdet_mom = 'gauss'
    # msk = mdet._make_mdet_cuts_wmom(
    #         d,
    #         min_s2n=5.0,
    #         min_t_ratio=1.1,
    #         n_terr=0.0,
    #         max_mfrac=0.1,
    #         max_s2n=np.inf,
    #     )
    noshear_mask = (d['mdet_step'] == 'noshear')
    d = d[msk & noshear_mask]
    # except:
    #     print('error cutting')
    #     continue
    
    # mdet_obj = np.zeros(len(d), dtype=[('tilename', object), ('slice_id', 'i2'), ('x', 'f8'), ('y', 'f8'), ('ra', 'f8'), ('dec', 'f8')]) 
    mdet_obj = np.zeros(len(d), dtype=[('ra', 'f8'), ('dec', 'f8'), ('s2n', 'f8'), ('gauss_T_ratio', 'f8'), ('g1', 'f8'), ('g2', 'f8'), ('psfrec_e1', 'f8'), ('gauss_T', 'f8'), ('psfrec_e2', 'f8'), ('g1_cov', 'f8'), ('g2_cov', 'f8'), ('R_all', 'f8'), ('w', 'f8'), ('mag_g', 'f8'), ('mag_r', 'f8'), ('mag_i', 'f8')])
    # mdet_obj = np.zeros(len(d), dtype=[('ra', 'f8'), ('dec', 'f8'), ('s2n', 'f8'), ('gauss_T_ratio', 'f8'),  ('pgauss_T', 'f8'), ('gauss_T', 'f8'), ('pgauss_T_err', 'f8'), ('gauss_T_err', 'f8'), ('mag_g', 'f8'), ('mag_r', 'f8'), ('mag_i', 'f8'), ('w', 'f8') ])
    # mdet_obj = np.zeros(len(d), dtype=[('ra', 'f8'), ('dec', 'f8'), ('s2n', 'f8'), ('g1', 'f8'), ('g2', 'f8'), ('x', 'f8'), ('y', 'f8'), ('g_flux', 'f8'), ('i_flux', 'f8'), ('z_flux', 'f8')])
    # shear_response, shear_wgt = _find_shear_weight(d, wgt_dict, 10, 1000, 0.5, 5.0, 20, mdet_mom)
    noise_shear_wgt = 1/(0.17**2 + 0.5*(d[mdet_mom+'_g_cov_1_1'] + d[mdet_mom+'_g_cov_2_2']))

    # mdet_obj['tilename'] = d['tilename']
    # mdet_obj['slice_id'] = d['slice_id']
    # mdet_obj['x'] = d['x']
    # mdet_obj['y'] = d['y']
    mdet_obj['ra'] = d['ra']
    mdet_obj['dec'] = d['dec']
    mdet_obj['s2n'] = d[mdet_mom+'_s2n']
    # mdet_obj['gmi'] = flux2mag(d['pgauss_band_flux_g']) - flux2mag(d['pgauss_band_flux_i'])
    # mdet_obj['imz'] = flux2mag(d['pgauss_band_flux_i']) - flux2mag(d['pgauss_band_flux_z'])
    mdet_obj['mag_g'] = flux2mag(d['pgauss_band_flux_g']) 
    mdet_obj['mag_r'] = flux2mag(d['pgauss_band_flux_r']) 
    mdet_obj['mag_i'] = flux2mag(d['pgauss_band_flux_i']) 
    # mdet_obj['z_flux'] = d['pgauss_band_flux_z']
    mdet_obj['g1'] = d[mdet_mom+'_g_1']
    mdet_obj['g2'] = d[mdet_mom+'_g_2']
    mdet_obj['g1_cov'] = d[mdet_mom+'_g_cov_1_1']
    mdet_obj['g2_cov'] = d[mdet_mom+'_g_cov_2_2']
    mdet_obj['R_all'] = R_shape_err
    # mdet_obj['R_noise'] = R_shape_err
    mdet_obj['w'] = noise_shear_wgt
    # mdet_obj['w_noise'] = noise_shear_wgt
    # mdet_obj['psf_e1'] = d[mdet_mom+'_psf_g_1']
    # mdet_obj['psf_e2'] = d[mdet_mom+'_psf_g_2']
    mdet_obj['psfrec_e1'] = d['psfrec_g_1']
    mdet_obj['psfrec_e2'] = d['psfrec_g_2']
    mdet_obj['gauss_T_ratio'] = d['gauss_T_ratio']
    # mdet_obj['pgauss_T'] = d['pgauss_T']
    # mdet_obj['pgauss_T_err'] = d['pgauss_T_err']
    mdet_obj['gauss_T'] = d['gauss_T_ratio'] * d['gauss_psf_T']
    # mdet_obj['gauss_T_err'] = d['gauss_T_err']
    
    # obj_list.append(mdet_obj)
    if i == 0:
        mdet_all = mdet_obj
    else:
        mdet_all = np.concatenate([mdet_all, mdet_obj])
# fio.write('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/v3/metadetection_v3_flat.fits', mdet_all)

200it [44:20, 13.30s/it]


In [ ]:
np.percentile(mdet_all['pgauss_T'], 75)

In [12]:
len(mdet_all)

150093691

In [11]:
fio.write('/pscratch/sd/m/myamamot/des-y6-analysis/y6_measurement/v5/metadetection_v5_flat_shape_err.fits', mdet_all)

In [9]:
max(mdet_all['w'])

31.3759296934466

# Analysis Choices

In [ ]:
d = fio.read('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/mean_shear_stats_final_v1.fits')

In [ ]:
fig,ax=plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ax[0].hist(np.log10(d['wmom_s2n']), bins=500, histtype='step')
ax[1].hist(d['wmom_T_ratio'], bins=500, histtype='step')
plt.show()

In [ ]:
print('S/N cut 300', np.mean((d['wmom_s2n'] < 300)))
print('S/N cut 500', np.mean((d['wmom_s2n'] < 500)))

In [ ]:
print('T_ratio cut 2.0', np.mean((d['wmom_T_ratio'] < 2.0)))

In [ ]:
d_v2_snweight = fio.read('/global/cscratch1/sd/myamamot/des-y6-analysis/y6_measurement/metadetection_v2_snweight.fits')

In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
h = ax.hist(d_v2_snweight['w'], histtype='step', bins=500)
wgt2 = 1/(0.07**2 + )
h = ax.hist(wgt2, histtype='step', bins=500)
ax.set_xlabel('per-galaxy weight')
plt.tight_layout()

In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
wgt = 1/(0.25**2 + 0.5*(d_v2_snweight['g1_cov'] + d_v2_snweight['g2_cov']))
h = ax.hist(wgt, histtype='step', bins=500)
ax.set_xlabel('per-galaxy weight')
plt.tight_layout()

# HDF5 file

In [6]:
import h5py as h5
f = h5.File('/global/cfs/cdirs/des/y6-shear-catalogs/Y6A2_METADETECT_V5a/metadetect_desdmv5a_cutsv5.h5', 'r')

In [52]:
d = f.get('/mdet/2p')

In [32]:
np.mean(d['ra'])

95.68175472498422

In [34]:
l = {'ra', 'dec', 'gauss_g_1', 'gauss_g_2'}

In [56]:
d[('ra', 'dec')]

AttributeError: 'tuple' object has no attribute 'encode'